## 카카오 내비 데이터 수집

In [1]:
# 관련 라이브러리를 호출합니다.
import requests
from urllib.parse import urlencode
import json
import pandas as pd
from datetime import timedelta

In [2]:
# 카카오 지도 API REST Key를 지정합니다.
apiKey = '8ffa6eab80c438b9504391dfb217f664'

### 위경도 좌표 얻기

In [3]:
# 도로명주소를 입력하면 위경도 좌표를 반환하는 함수를 생성합니다.
def getCoord(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    qry = dict(query = addr)
    key = apiKey
    auth = f'KakaoAK {apiKey}'
    
    res = requests.get(
        url = url, 
        params = qry, 
        headers = dict(Authorization = auth)
    )
    
    dat = json.loads(s = res.text)
    return dat['documents'][0]['address']['x'], dat['documents'][0]['address']['y']

In [4]:
# 출발지 도로명주소로 위경도 좌표를 얻습니다.
getCoord(addr = '서울 중구 다산로 149')

('127.011666505681', '37.5570916390478')

In [5]:
# 도착지 도로명주소로 위경도 좌표를 얻습니다.
getCoord(addr = '경기 양평군 강하면 강남로 316')

('127.421778888836', '37.5003089274379')

### 두 지역의 네비 결과 얻기

In [6]:
# 출발지와 도착지 위경도 좌표로 이동거리와 소요시간을 반환하는 함수를 생성합니다.
def getNavi(origin, destination):
    url = 'https://apis-navi.kakaomobility.com/v1/directions'

    qry = dict(
        origin = ','.join(getCoord(addr = origin)),
        destination = ','.join(getCoord(addr = destination))
    )

    urlencode(query = qry, safe = ',')
    
    auth = f'KakaoAK {apiKey}'

    res = requests.get(
        url = url, 
        params = urlencode(query = qry, safe = ','),
        headers = dict(Authorization = auth)
    )
    
    dat = json.loads(s = res.text)
    dis = f"{dat['routes'][0]['summary']['distance'] / 1000}km"
    sec = str(timedelta(seconds = dat['routes'][0]['summary']['duration']))
    return dis, sec

In [7]:
# 출발지와 도착지 도로명주소를 입력하고 내비 결과를 얻습니다.
getNavi(origin = '서울 중구 다산로 149', 
        destination = '경기 양평군 강하면 강남로 316')

('53.235km', '1:17:33')

## End of Document